In [184]:
# Load all required libraries for the notebook, including data package
if(!require("opendatatoronto")) install.packages("opendatatoronto");    library(opendatatoronto) # data source
if(!require("tidyjson")) install.packages("tidyjson");    library(tidyjson)
if(!require("tidygeocoder")) install.packages("tidygeocoder");    library(tidygeocoder)
if(!require("sf"))   install.packages("sf");    library(sf)
if(!require("ggspatial")) install.packages("ggspatial");    library(ggspatial)
if(!require("stringr")) install.packages("stringr");    library(stringr)
if(!require("dplyr")) install.packages("dplyr");    library(dplyr)
if(!require("ggplot2")) install.packages("ggplot2");    library(ggplot2)
if(!require("tidyverse")) install.packages("tidyverse");    library(tidyverse)
if(!require("FNN")) install.packages("FNN");    library(FNN)
if(!require("terra")) install.packages("terra");    library(terra)
if(!require("mapview")) install.packages("mapview");    library(mapview)
if(!require("forecast")) install.packages("forecast");    library(forecast)
if(!require("purrr")) install.packages("purrr"); library(purrr)
if(!require("e1071")) install.packages("e1071");    library(e1071)


In [185]:
# set RUN_DATA_PROCESSING to FALSE by default as all the data processing has been done to speed up the code run time
# set it to TRUE if you are want to run the code to process the raw data from scratch. 
# Or if the pre-processed data files are not available in ../Data/DataProcessing/
RUN_DATA_PROCESSING <- FALSE

if(RUN_DATA_PROCESSING) {
    print("RUN_DATA_PROCESSING is TRUE")
} else {
    print("RUN_DATA_PROCESSING is FALSE")
}


[1] "RUN_DATA_PROCESSING is FALSE"


In [169]:
## Get Data 1 - Traffic


# # load data # The method of loading data directly using turns out to be insufficient as the get_resource() 
# only returns first 32000 rows of record.

# To Run the code download the raw files from 
# https://open.toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/ 
# and save the files in .csv format to the path: ../Data/Toronto/Traffic, 7 Files below. 
# count_metadata.csv
# locations.csv
# raw-data-1980-1989.csv
# raw-data-1990-1999.csv
# raw-data-2000-2009.csv
# raw-data-2010-2019.csv
# raw-data-2020-2029.csv

# Dataframe with all intersection and average traffic per year per intersection from 1984 to 2023. 


if(RUN_DATA_PROCESSING) {
    print("RUN_DATA_PROCESSING is TRUE")
    location <- read.csv("../Data/Toronto/Traffic/locations.csv") # ensure these path are correct if you have to download the files manually. 
    traffic1 <- read.csv("../Data/Toronto/Traffic/raw-data-1980-1989.csv")
    traffic2 <- read.csv("../Data/Toronto/Traffic/raw-data-1990-1999.csv")
    traffic3 <- read.csv("../Data/Toronto/Traffic/raw-data-2000-2009.csv")
    traffic4 <- read.csv("../Data/Toronto/Traffic/raw-data-2010-2019.csv")
    traffic5 <- read.csv("../Data/Toronto/Traffic/raw-data-2020-2029.csv")
    all_traffic = bind_rows(traffic1,traffic2,traffic3,traffic4,traffic5) # combine all raw data into one data frame

    # clean and transform load - Traffic Data
    # Output data for modelling CleanTraffic


    # get full intersection volume for each intersection based on peak hours of each day. 
    # get average vol per intersection per year. 
    # get number of years list, sort from low to high




    # define parameters for cleaning

    peakhours <- 4 # number of peak hours of data per day. value should be between 1 and 10

    CleanTraffic <- all_traffic %>%
      filter(centreline_type == 2) %>% # only need intersection data
      mutate(counthour = str_extract(time_start, "(?<=T)(\\d+)(?=\\:)")) %>% # extract hour
      mutate(total_int_traffic = sb_cars_r + sb_cars_t + sb_cars_l +
        nb_cars_r + nb_cars_t + nb_cars_l + wb_cars_r + wb_cars_t +
        wb_cars_l + eb_cars_r + eb_cars_t + eb_cars_l) %>% # get total sum
      select(one_of(c(
        "count_date", "location_id", "location", "lng", "lat", "counthour",
        "total_int_traffic"
      ))) %>% # remove raw attributes, retain aggregate only
      group_by(across(all_of(c("count_date", "location_id", "location", "lng", "lat", "counthour")))) %>%
      summarise_at("total_int_traffic",sum) %>% # agregate hourly volume
      group_by(across(all_of(c("count_date", "location_id", "location", "lng", "lat")))) %>%
      slice_max(order_by = total_int_traffic, n = peakhours) %>% # filter top peak hour volume
      group_by(across(all_of(c("count_date", "location_id", "location", "lng", "lat")))) %>%
      summarise_at("total_int_traffic", sum)%>% # aggregate daily peak hour volume
      mutate(count_date= as.Date(count_date, format("%Y-%m-%d"))) %>% 
      mutate(year = as.numeric(format(count_date,'%Y'))) %>% #add year
      group_by(across(all_of(c("year", "location_id", "location", "lat","lng")))) %>% # group by year to get the average per intersection per year
      summarise(AvgTotal = mean(total_int_traffic), .groups = "drop")  # average traffic volume for that year and location
    traffic_years <- unique(CleanTraffic$year) # get number of years list, sort from low to high

    # save CleanTraffic as a csv file for ease of future use
    write.csv(CleanTraffic, "../Data/DataProcessing/CleanTraffic.csv")
} else {
    print("RUN_DATA_PROCESSING is FALSE")
    CleanTraffic = read.csv("../Data/DataProcessing/CleanTraffic.csv")
    traffic_years <- unique(CleanTraffic$year)
}
head(CleanTraffic) # data preview

[1] "RUN_DATA_PROCESSING is FALSE"


,X,year,location_id,location,lat,lng,AvgTotal
,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>
1,1,1984,3926,BLOOR ST AT SOUTH KINGSWAY & RIVERVIEW GARDENS (PX 334),43.64831,-79.48575,8474
2,2,1984,3934,LAKE SHORE BLVD W AT LONG BRANCH AVE (PX 240),43.59436,-79.53402,5647
3,3,1984,3940,BLACK CREEK DR AT LAWRENCE AVE W (PX 1348),43.70495,-79.49410,17625
4,4,1984,3950,KINGSTON RD AT BROOKLAWN AVE & ST CLAIR AVE E,43.72203,-79.23602,11820
5,5,1984,3953,KINGSTON RD AT LAWRENCE AVE (PX 144),43.76765,-79.18961,13885
6,6,1984,3968,FINCH AVE AT WESTON RD (PX 589),43.75209,-79.54238,12023


In [161]:



## Get Data 2 - Business
    
# output Data Description: 
# the output dataframe is 'biz_geo_loc1' which contains 13 variables i.e.
# Category : business type as defined from Opendatatoronto
# Operating.Name: business name as registered in Opendatatoronto
# city: City where the business is located (extracted from business address)
# Zip: postal code where business is located (extracted from business address)
# Street street name where business is located (extracted from business address)
# lat, long : Lattitude , Longitude converting from address into geocoder
# lamda : customer arrival rate used to simulate the number of customer during busy hours(assumption)
# mu : expected time spent (in hr) during busy hours of a customer (exponential distribution) 
# qCustomer: number of customer at busy hours (simulated from lamda)
# tCustomer: time spent at the business per customer (simulated)
##########################THE CODES ######################################
# codes are separated into few sections:
#section a) load data from Opendatatoronto to biz_data dataframe
#section b) data cleansing in preparation for geocoder conversion
#section c) address conversion to geocoder 
#section d) adding simulated number of customer (qCustomer) and time spent (tCustomer) during busy hours
##########################################################################
####section a) load data from Opendatatoronto to biz_data dataframe
# Load libraries needed for loading data from toronto open data source

if(RUN_DATA_PROCESSING) {
    print("RUN_DATA_PROCESSING is TRUE")
    
    if (!require(opendatatoronto)) install.packages("opendatatoronto")
    library(opendatatoronto)





    # get package
    package <- show_package("57b2285f-4f80-45fb-ae3e-41a02c3a137f")
    #package

    # get all resources for this package
    resources <- list_package_resources("57b2285f-4f80-45fb-ae3e-41a02c3a137f")

    # identify datastore resources; by default, Toronto Open Data sets datastore resource format to CSV for non-geospatial and GeoJSON for geospatial resources
    datastore_resources <- filter(resources, tolower(format) %in% c('csv', 'geojson'))

    # load the first datastore resource as a sample
    biz_data <- filter(datastore_resources, row_number()==1) %>% get_resource()
    #biz_data

    ### backup data to biz_data_bak
    biz_data_bak = biz_data

    ###Meaning of the data according to the readme from website are as follows
    ###Category=Type of licence or permit
    ###Licence No=Number. of licence issued by City of Toronto
    ###Operating Name=Name that company operates under
    ###Issued=Date of issue of licence/permit
    ###Client Name=Name that the licence is issued to
    ###Business Phone=Client Business Phone Number
    ###Business Phone Ext.=Client Business Phone Extension Number
    ###Licence Address Line 1=First line of client's business address
    ###Licence Address Line 2=Client address town and province
    ###Licence Address Line 3=Client address postal code
    ###Conditions=Restriction on the licence/permit recorded in the licensing system
    ###Free Form Conditions Line 1=Restriction/comment on the licence/permit
    ###Free Form Conditions Line 2=Continuation of line 1 
    ###Plate No.=Licence identifying plate, issued to most vehicles
    ###Endorsements=Activity permitted under the licence
    ###Cancel Date=Date the license or permit was cancelled
    ############### End Of Data loading from  Toronto Open Data Source########
    ##########################################################################
    ####section b) data cleansing in preparation for geocoder conversion
    ### Start cleaning up with change data type, filled in any empty data in Operating Name
    biz_data$Category = as.factor(biz_data$Category)
    biz_data$`Licence Address Line 2` = as.factor(biz_data$`Licence Address Line 2`)
    biz_data$Issued = as.Date(biz_data$Issued)
    biz_data$`Cancel Date` = as.Date(biz_data$`Cancel Date`)
    biz_data$`Last Record Update` = as.Date(biz_data$`Last Record Update`)
    biz_data$'Operating Name' <-ifelse(biz_data$'Operating Name' == "",biz_data$'Client Name',biz_data$'Operating Name')
    #str(biz_data)
    #summary(biz_data)
    #head(biz_data)

    ### remove biz that have cancel date as it indicates that business has already been cancelled (i.e. terminated)
    biz_data = biz_data[which(is.na(biz_data$'Cancel Date')),]

    #################################################################################################
    ### only select business in Toronto with zip code starting with M3, M4, M5 and M6
    biz_data = biz_data[which(biz_data$'Licence Address Line 2' == "TORONTO, ON"),]
    biz_data = biz_data[which(biz_data$'Licence Address Line 3' > "M3" & biz_data$'Licence Address Line 3' < "M7"),]
    #################################################################################################
    #biz_data = biz_data %>%
    #  select(-'Cancel Date')

    ###make column to identify street name from Licence.Address.Line.1 (might be useful for cluster work later)
    ###first extracting the address number. grab only the data that are after the first space
    biz_data$Street <- sub("^\\S+\\s+",'', biz_data$'Licence Address Line 1')
    ###first extracting the address number. remove number
    biz_data$Street <- gsub("[[:digit:]]", "", biz_data$Street)
    ###remove other unneccessary characters
    biz_data$Street <- sub('-  ', '', biz_data$Street)
    biz_data$Street <- sub('&A ', '', biz_data$Street)
    biz_data$Street <- sub('& A ', '', biz_data$Street)
    biz_data$Street <- sub('&B ', '', biz_data$Street)
    biz_data$Street <- sub('& B ', '', biz_data$Street)
    biz_data$Street <- sub('&  ', '', biz_data$Street)
    biz_data$Street <- sub('&& ', '', biz_data$Street)
    biz_data$Street <- sub('& ', '', biz_data$Street)
    biz_data$Street <- sub('/ ', '', biz_data$Street)
    biz_data$Street <- sub('-A ', '', biz_data$Street)
    biz_data$Street <- sub('A-', '', biz_data$Street)
    biz_data$Street <- sub('- A ', '', biz_data$Street)
    biz_data$Street <- sub('-B ', '', biz_data$Street)
    biz_data$Street <- sub('- B ', '', biz_data$Street)
    biz_data$Street <- sub("(?<=^.{0}),", "", biz_data$Street, perl = TRUE )
    biz_data$Street <- sub("(?<=^.{0})&", "", biz_data$Street, perl = TRUE )
    biz_data$Street <- sub("(?<=^.{0})/", "", biz_data$Street, perl = TRUE )
    biz_data$Street <- sub("(?<=^.{0})-", "", biz_data$Street, perl = TRUE )
    ### as data is not clean, there are sometimes suite number or other suffix follow them. therefore, we clean that part up too.
    biz_data$Street <- sub(", #.*", '', biz_data$Street)
    biz_data$Street <- sub(" #.*", '', biz_data$Street)
    biz_data$Street <- sub("#.*", '', biz_data$Street)
    biz_data$Street <- sub(" ,.*", '', biz_data$Street)
    biz_data$Street <- sub(",.*", '', biz_data$Street)

    ### use similar code to clean data and prepare for geo location
    biz_data$StreetAddress = paste(biz_data$'Licence Address Line 1')
    ###remove other characters from address one
    biz_data$StreetAddress <- sub('-  ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('&A ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('& A ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('&B ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('& B ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('&  ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('&& ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('& ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('/ ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('-A ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('A-', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('- A ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('-B ', '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub('- B ', '', biz_data$StreetAddress)
    ### Removing suite number or other suffix follow them. therefore
    biz_data$StreetAddress <- sub(", #.*", '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub(" #.*", '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub("#.*", '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub(" ,.*", '', biz_data$StreetAddress)
    biz_data$StreetAddress <- sub(",.*", '', biz_data$StreetAddress)
    ############################### End of Data Cleansing ####################
    ##########################################################################
    ####section c) address conversion to geocoder 
    ###convert to Lat- Long using tidygeocode : Reference https://www.youtube.com/watch?v=7nFZ5BwkAXc
    ### Load libraries needed for geocode
    if (!require(tidyverse)) install.packages("tidyverse")
    library(tidyverse)  
    if (!require(tidygeocoder)) install.packages("tidygeocoder")
    library(tidygeocoder)  
    if (!require(sf)) install.packages("sf")
    library(sf) 
    # if (!require(mapview)) install.packages("mapview")
    # library(mapview) 

    ### Prepare Data to be converted
    biz_data$Address <- paste(biz_data$StreetAddress,biz_data$'Licence Address Line 2',biz_data$'Licence Address Line 3', sep = ",")
    biz_geo_loc_a = biz_data %>%
      select(Category,`Operating Name`,city = 'Licence Address Line 2',Zip = 'Licence Address Line 3',Street, Address)

    ### Convert address to lat - long
    biz_geo_loc = biz_geo_loc_a %>% filter(str_detect(Zip, "M4"))%>%
      tidygeocoder:: geocode(address = Address, method = "osm")
    biz_geo_loc1 = biz_geo_loc[-which(is.na(biz_geo_loc$lat)),]
    biz_geo_loc2 = biz_geo_loc[which(is.na(biz_geo_loc$lat)),]

    biz_geo_loc = biz_geo_loc_a %>% filter(str_detect(Zip, "M5"))%>%
      tidygeocoder:: geocode(address = Address, method = "osm")
    biz_geo_loc1 = rbind(biz_geo_loc1,biz_geo_loc[-which(is.na(biz_geo_loc$lat)),])
    biz_geo_loc2 = rbind(biz_geo_loc2,biz_geo_loc[which(is.na(biz_geo_loc$lat)),])

    biz_geo_loc = biz_geo_loc_a %>% filter(str_detect(Zip, "M6"))%>%
      tidygeocoder:: geocode(address = Address, method = "osm")
    biz_geo_loc1 = rbind(biz_geo_loc1,biz_geo_loc[-which(is.na(biz_geo_loc$lat)),])
    biz_geo_loc2 = rbind(biz_geo_loc2,biz_geo_loc[which(is.na(biz_geo_loc$lat)),])

    biz_geo_loc = biz_geo_loc_a %>% filter(str_detect(Zip, "M3"))%>%
      tidygeocoder:: geocode(address = Address, method = "osm")
    biz_geo_loc1 = rbind(biz_geo_loc1,biz_geo_loc[-which(is.na(biz_geo_loc$lat)),])
    biz_geo_loc2 = rbind(biz_geo_loc2,biz_geo_loc[which(is.na(biz_geo_loc$lat)),])

    biz_geo_loc = biz_geo_loc_a %>% filter(str_detect(Zip, "M7"))%>%
      tidygeocoder:: geocode(address = Address, method = "osm")
    biz_geo_loc1 = rbind(biz_geo_loc1,biz_geo_loc[-which(is.na(biz_geo_loc$lat)),])
    biz_geo_loc2 = rbind(biz_geo_loc2,biz_geo_loc[which(is.na(biz_geo_loc$lat)),])

    ###check if any na
    sapply(biz_geo_loc1, function(x) sum(is.na(x)))

    # write.csv(biz_geo_loc1, "~/biz_geo_loc1.csv")




    ###display on map # only works in R not Jupyter
    biz_geo_loc_sf = biz_geo_loc1 %>%
      st_as_sf(
        coords = c("long","lat"),
        crs = 4326
      )

    biz_geo_loc_sf %>% mapview()

    ##########################################################################
    ##########################################################################
    ####section d) adding simulated number of customer (qCustomer) and time spent (tCustomer) during busy hours

    biz_cat1 = data.frame(cbind(
      c("RETAIL STORE (FOOD)","HOLISTIC CENTRE","DRIVING SCHOOL OPERATOR (B)",	
        "DRIVE-SELF RENTAL OWNER","PLACE OF AMUSEMENT",	"PRIVATE PARKING ENFORCEMENT AGENCY",	
        "SMOKE SHOP",	"BILLIARD HALL","BODY RUB PARLOUR",	"TAXICAB BROKER",	"BOWLING HOUSE",	
        "ADULT ENTERTAINMENT CLUB",	"LIMOUSINE SERVICE COMPANY",	
        "PRIVATE TRANSPORTATION COMPANY",	"CARNIVAL",	"TEMPORARY SIGN PROVIDER",
        "TAXICAB OPERATOR",	"SHORT TERM RENTAL COMPANY","CIRCUS"), 
      c(30,5,5,10,30,5,10,20,10,10,20,5,10,10,3,3,10,3,100)) , 
      c(1,1.5,1,0.33,1,0.33,0.5,1,1,0.33,1,1,0.33,0.33,0.33,0.33,0.33,0.33,2) ,
      c(0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5)
      )
    colnames(biz_cat1) = c("Category","lamda","mu","sigma")
    biz_cat1$lamda = as.integer(biz_cat1$lamda)
    biz_cat1$mu = as.numeric(biz_cat1$mu)
    biz_cat1$sigma = as.numeric(biz_cat1$sigma)

    biz_customer = as.data.frame(biz_cat1)

    biz_geo_loc1 = left_join(biz_geo_loc1,biz_customer, by = "Category")

    # for reproducibility but mask it when want to do simulation
    set.seed(138)
    k = nrow(biz_geo_loc1)
    biz_geo_loc1 = data.frame(Category = biz_geo_loc1$Category, Operating.Name = biz_geo_loc1$'Operating.Name',
                              city = biz_geo_loc1$city, Zip = biz_geo_loc1$Zip,  Street =biz_geo_loc1$Street,
                              Address = biz_geo_loc1$Address, lat = biz_geo_loc1$lat,long = biz_geo_loc1$long,
                              lamda =biz_geo_loc1$lamda,  mu = biz_geo_loc1$mu, sigma = biz_geo_loc1$sigma,
                              qCustomer = rpois(k,biz_geo_loc1$lamda),
                              tCustomer = rexp(k,biz_geo_loc1$mu)) 
   
           
    #review qCustomer * tCustomer summary
    qCustomer_input_summary = biz_geo_loc1 %>%                            
      group_by(Category = biz_geo_loc1$Category, lamda = biz_geo_loc1$lamda ) %>%
      summarize(n = n(), min = min(qCustomer), q1 = quantile(qCustomer, 0.25), median = median(qCustomer), mean = mean(qCustomer), q3 = quantile(qCustomer, 0.75), max = max(qCustomer))
    #qCustomer_input_summary

    tCustomer_input_summary = biz_geo_loc1 %>%                            
      group_by(Category = biz_geo_loc1$Category,mu = biz_geo_loc1$mu) %>%
      summarize(n = n(), min = min(tCustomer), q1 = quantile(tCustomer, 0.25), median = median(tCustomer), mean = mean(tCustomer), q3 = quantile(tCustomer, 0.75), max = max(tCustomer))
    #tCustomer_input_summary
    write.csv(biz_geo_loc1, "../Data/DataProcessing/biz_geo_loc1.csv")
##########################################################################
##########################################################################
} else {
    print("RUN_DATA_PROCESSING is FALSE")
    biz_geo_loc1 <- read.csv("../Data/DataProcessing/biz_geo_loc1.csv")
}
head(biz_geo_loc1) # data preview

[1] "RUN_DATA_PROCESSING is FALSE"


,X,Category,Operating.Name,city,Zip,Street,Address,lat,long,lamda,mu,sigma,qCustomer,tCustomer
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>
1,1,TAXICAB OPERATOR,NORTHLAND TAXI,"TORONTO, ON",M4H 1H2,THORNCLIFFE PARK DR,"2 THORNCLIFFE PARK DR,TORONTO, ON,M4H 1H2",43.70593,-79.35204,10,0.33,0.5,10,1.6406388
2,2,PRIVATE PARKING ENFORCEMENT AGENCY,HALO SECURITY INC,"TORONTO, ON",M4L 1G1,QUEEN ST E,"1574 QUEEN ST E,TORONTO, ON,M4L 1G1",43.66607,-79.31836,5,0.33,0.5,6,0.8364520
3,3,PRIVATE PARKING ENFORCEMENT AGENCY,MERIDIAN SECURITY SOLUTIONS INC,"TORONTO, ON",M4N 3N1,YONGE ST,"3080 YONGE ST,TORONTO, ON,M4N 3N1",43.72560,-79.40252,5,0.33,0.5,2,8.7843074
4,4,DRIVING SCHOOL OPERATOR (B),ALBERT DRIVING SCHOOL,"TORONTO, ON",M4C 1L7,DANFORTH AVE,"2704 DANFORTH AVE,TORONTO, ON,M4C 1L7",43.68906,-79.29839,5,1.00,0.5,2,0.6605323
5,5,DRIVING SCHOOL OPERATOR (B),LEASIDE DRIVING ACADEMY,"TORONTO, ON",M4G 3C1,BAYVIEW AVE,"1659 BAYVIEW AVE,TORONTO, ON,M4G 3C1",43.70734,-79.37562,5,1.00,0.5,4,0.2136136
6,6,DRIVING SCHOOL OPERATOR (B),NEW METHOD DRIVING SCHOOL,"TORONTO, ON",M4M 1Y5,GERRARD ST E,"737 GERRARD ST E,TORONTO, ON,M4M 1Y5",43.66644,-79.34743,5,1.00,0.5,7,0.7268105


In [164]:
## Get data 3 - Parking Data (GreenPParking) 
# Output:
# address : Address of the parking spot
# lat : Latitude of parking spot
# lng : Longitude of parking spot
# half_hr : rate of parking/half hr
# capacity : capacity of parking spot

## Cleaning Green P Parking

if(RUN_DATA_PROCESSING) {
    print("RUN_DATA_PROCESSING is TRUE")

    # Get Green P Parking package from Open Data-Toronto
    package <- show_package("b66466c3-69c8-4825-9c8b-04b270069193")

    data=as.data.frame('Green P Parking')  # read the dataset Green P Parking in the package 
    data<-show_package(package)

    resources<-list_package_resources(package)

    # Identify resources
    data_resources <- filter(resources, tolower(format) %in% c("csv", "json"))

    # Load Green P Parking data
    data <- filter(data_resources, row_number() == 1) %>% get_resource()

    # Extract required columns from main data
    data<- data.frame(id = data$carparks$id,
                      address=data$carparks$address,
                                         lat=data$carparks$lat,
                                         lng=data$carparks$lng,
                                          carpark_type=data$carparks$carpark_type_str,
                                          rate_half_hr=data$carparks$rate_half_hour,
                                          capacity=data$carparks$capacity,
                                          rate=data$carparks$rate_half_hour
                      )

    # Check class of each attribute
    sapply(data, class) 

    # Convert char to numeric class
    data$lat<-as.numeric(data$lat)
    data$lng<-as.numeric(data$lng)
    data$rate_half_hr<-as.numeric(data$rate_half_hr)
    data$capacity<-as.numeric(data$capacity)

    sapply(data, class) 

    # Check for missing values
    any(is.na(data))

    # Extract street name from address
    data <- data %>%
      mutate(extracted_address = str_replace_all(data$address, "\\(.*?\\)",""))

    data$extracted_address<-str_replace_all(data$extracted_address, "-.*", "")
    data$extracted_address<-str_replace_all(data$extracted_address, ",.*", "")


    data <- data %>%
      filter(carpark_type == "Surface")

    #save data the car parking as a csv file for ease of future use
    write.csv(data, "../Data/DataProcessing/carpark_Surface_data.csv")
    
} else {
    print("RUN_DATA_PROCESSING is FALSE")
    data <- read.csv("../Data/DataProcessing/carpark_Surface_data.csv") # read from pre-processed data 
}
head(data) # data preview of the surface carpark data

[1] "RUN_DATA_PROCESSING is FALSE"


,X,id,address,lat,lng,carpark_type,rate_half_hr,capacity,rate,extracted_address
,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,1,3,13 Isabella Street,43.66758,-79.38471,Surface,3.00,33,3.00,13 Isabella Street
2,2,5,15 Wellesley Street East,43.66484,-79.38359,Surface,3.00,135,3.00,15 Wellesley Street East
3,3,12,30 Alvin Avenue,43.68919,-79.39270,Surface,3.50,188,3.50,30 Alvin Avenue
4,4,17,716 Pape Avenue,43.67971,-79.34538,Surface,1.75,70,1.75,716 Pape Avenue
5,5,18,351 Keele Street,43.66458,-79.46399,Surface,1.50,77,1.50,351 Keele Street
6,6,19,385 Pacific Avenue,43.66474,-79.46812,Surface,1.50,71,1.50,385 Pacific Avenue


In [14]:
## Pre-Processing 1 : Use KNN to get near by historic traffic for each parking lot

# Combine Parking Data and Clean Traffic Data to prepare input data for Time Series Model to predict traffic / EV charging demand

# loop through low to high year number
# filter each year, to list of unique intersections with traffic volume
# run knn algo against parking lot, find k nearst intersection
# use knn nn.index to find the k intersection vol, get average, creating a vector
# add vector to parking data as new column with year and volume
# next loop for the next year

#Output TS_Input

if(RUN_DATA_PROCESSING) {
    print("RUN_DATA_PROCESSING is TRUE")


    K <- 8 # Parameter used in knn calculation. 
    All_park <- data %>%
        select(one_of(c("lat","lng"))) # location coordinates of all surface car parks. 

    TS_Input <- data # TS_Input is the input data for the time series of traffic volume forecast. 

    for (i in traffic_years) {
        # print(paste("This is year" ,i))
        year_traffic<- filter(CleanTraffic, year == i) # get the traffic volume, intersection for each year.
        All_int <-select(year_traffic,one_of(c("lat","lng"))) # extract just the coordinates for K nearest neighter calculation
        knn_dist <- get.knnx(All_int, All_park, k=K, algorithm="kd_tree") # get the K nearest intersection to the parking lot and their index list

        get.mean <- function(x) { # custom function defined to calculate the average of the K nearest intersection volume
        mean(slice(year_traffic, c(x))$AvgTotal)
        }

        TS_Input[paste0("NearbyTraffic_",i)]  <- apply(knn_dist$nn.index,1,get.mean) # calculate the avaerage per row of indices

    }

    #save TS_Input as a csv file for ease of future anaylsis. 
    write.csv(TS_Input, "../Data/DataProcessing/TS_Input.csv")

} else {
    print("RUN_DATA_PROCESSING is FALSE")
    TS_Input <-read.csv("../Data/DataProcessing/TS_Input.csv") # read from pre-processed data
}
head(TS_Input) # data preview

In [165]:
# Pre-Processing 2 - Get 2022 and 2024 ( forecasted) EV traffic data near each parking lot
# 
# Time Series Forecast traffic for 2024 given the TS_Input
# apply % of EV vehicle multiplier for 2022 and predicted year 2024 to get amount of estimated EV traffic.
# multiplier % based on assumption 30% of all car traffic ( based on  website) will be EV by 2030 and in 2022 3% of all car 
# traffic are EV. ( based on website). Assumping constant rate of increase from 2022 to 2030, i.e. 30% y.o.y increase
# data frame that has the original location, EV traffic for 2022, and predicted EV traffic for 2024


library(forecast)
library(purrr)


if(RUN_DATA_PROCESSING) {
    print("RUN_DATA_PROCESSING is TRUE")


    # Time series model and Forecast for 2024
    arima_models <- apply(TS_Input[, -c(1,2,3,4,5,6,7,8)], 1, auto.arima)
    model_forecasts_2024 <- lapply(arima_models, function(x) forecast(x, h = 1))

    TS_Output_2024 <- map_dbl(model_forecasts_2024, "mean")
    TS_Output_2024 <- round(TS_Output_2024)


    # Format output for 2024
    # Data frame that contains the original location, lat, and lng columns,
    # % of EV traffic for 2022 and % of EV traffic for predicted 2024

    # % of EV vehicle multiplier for 2022
    mulitplier_2022 = .03

    # % of EV Vehicle multiplier for predicted year 
    multiplier_predicted = .0533

    TS_Output_df = data.frame(TS_Input$address,TS_Input$lat,TS_Input$lng, TS_Input$NearbyTraffic_2022,TS_Output_2024)
    #head(TS_Output_df)

    # Apply % of EV Vehicle multiplier to 2022 and predicted year 2024, present a new columns to Df
    traffic_2022 <- data.frame(TS_Output_df$TS_Input.NearbyTraffic_2022)
    traffic_2024 <-data.frame(TS_Output_df$TS_Output_2024)

    TS_Output_df$EV_nearbyTraffic_2022 <- apply(traffic_2022, 1, function(x) round(x * mulitplier_2022))
    TS_Output_df$EV_nearbyTraffic_2024 <- apply(traffic_2024, 1, function(x) round(x * multiplier_predicted))




    # save TS_Output_df as a csv file for ease of future use
    write.csv(TS_Output_df, "../Data/DataProcessing/TS_Output.csv")    

} else {
    print("RUN_DATA_PROCESSING is FALSE")
    TS_Output_df <-read.csv("../Data/DataProcessing/TS_Output.csv") # read from pre-processed data
}      
                                            
# Final Output
head(TS_Output_df)  # data preview

[1] "RUN_DATA_PROCESSING is FALSE"


,X,TS_Input.address,TS_Input.lat,TS_Input.lng,TS_Input.NearbyTraffic_2022,TS_Output_2024,EV_nearbyTraffic_2022,EV_nearbyTraffic_2024
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
1,1,13 Isabella Street,43.66758,-79.38471,4848.616,4497,145,240
2,2,15 Wellesley Street East,43.66484,-79.38359,5281.462,5540,158,295
3,3,30 Alvin Avenue,43.68919,-79.39270,4521.144,4811,136,256
4,4,716 Pape Avenue,43.67971,-79.34538,4912.333,3750,147,200
5,5,351 Keele Street,43.66458,-79.46399,6077.057,4739,182,253
6,6,385 Pacific Avenue,43.66474,-79.46812,5776.635,5188,173,277


In [168]:
## Pre-Processing 3 : Add new column Convert to indicate if parking lot has EV chargers installed for each lot

if(RUN_DATA_PROCESSING) {
    print("RUN_DATA_PROCESSING is TRUE")

# Add convert column to GreenPParking dataset
library(sf)

# Extracting address, latitude, and longitude
data <- data[, c("id","address", "lat", "lng","capacity")]

# Adding the new rows which represent parking lots with EV chargers already installed. 
# This information is based on the lastest EV Parking location update from the TPA Greep P website. 
# https://parking.greenp.com/ev-charging/
# These parking lots are considred high priority lots, there are 9 of these in total. 
new_rows <- data.frame(
  address = c(
    "365 Lippincott Street",
    "35 Erindale Avenue",
    "14 Arundel Avenue",
    "265 Armadale Avenue",
    "1612 Danforth Avenue",
    "117 Hammersmith Avenue",
    "166 Woodbine Ave",
    "19 Spadina Road",
    "2300 Lake shore Boulevard West",
    "265 Armadale Avenue",
    "2300 Lake shore Boulevard West"
  ),
  lat = c(
    43.665054,
    43.688543,
    43.695882,
    43.721568,
    43.684903,
    43.693819,
    43.674124,
    43.677754,
    43.623545
  ),
  lng = c(
    -79.409662,
    -79.411305,
    -79.42134,
    -79.427191,
    -79.325679,
    -79.428246,
    -79.319325,
    -79.403948,
    -79.479056
  )
)

# Adding the new rows to the extracted GreenPParking dataset
# data <- rbind(data, new_rows)

# Creating the 'Convert' column
data$Convert <- ifelse(data$address %in% new_rows$address, 1, 0)

# save the updated data df as a csv file for ease of future use
write.csv(data, "../Data/DataProcessing/carpark_Training_Data.csv")   
} else {
    print("RUN_DATA_PROCESSING is FALSE")
    data <- read.csv("../Data/DataProcessing/carpark_Training_Data.csv") # read data from pre-processed files
}
head(data) # preview data

[1] "RUN_DATA_PROCESSING is FALSE"


,X,id,address,lat,lng,capacity,Convert
,<int>,<int>,<chr>,<dbl>,<dbl>,<int>,<int>
1,1,3,13 Isabella Street,43.66758,-79.38471,33,0
2,2,5,15 Wellesley Street East,43.66484,-79.38359,135,0
3,3,12,30 Alvin Avenue,43.68919,-79.39270,188,0
4,4,17,716 Pape Avenue,43.67971,-79.34538,70,0
5,5,18,351 Keele Street,43.66458,-79.46399,77,0
6,6,19,385 Pacific Avenue,43.66474,-79.46812,71,0


In [ ]:
# ############################################## METHOD 1 - SVM (Without KNN) ##################################################

# # Read data
# parking<-data
# business<-biz_geo_loc1

# parking_sf <- st_as_sf(parking, coords = c("lng", "lat"), crs = 4326)

# # Convert the business dataset to a spatial object
# business_sf <- st_as_sf(business, coords = c("long", "lat"), crs = 4326)

# # Convert the parking dataset to a spatial object
# parking_sf <- st_as_sf(parking, coords = c("lng", "lat"), crs = 4326)

# # Perform a spatial join to find the nearest business for each parking space
# nearest_business <- st_nearest_feature(parking_sf, business_sf)
# nearest_points <-st_nearest_points(parking_sf,business_sf)

# # Add the nearest business information to the parking dataset
# parking_data_with_nearest_business <- cbind(parking, nearest_business)

# # Function to calculate angular distance between two points on Earth
# haversine_distance <- function(lon1, lat1, lon2, lat2) {
#   R <- 6371 # Earth radius in km
#   dlat <- (lat2 - lat1) * pi / 180
#   dlon <- (lon2 - lon1) * pi / 180
#   a <- sin(dlat/2)^2 + cos(lat1 * pi / 180) * cos(lat2 * pi / 180) * sin(dlon/2)^2
#   c <- 2 * atan2(sqrt(a), sqrt(1 - a))
#   distance <- R * c
#   return(distance) # Distance in km
# }
# parking_data_with_nearest_business$traffic_volume=TS_Output_df$EV_nearbyTraffic_2022	


# # # Convert Lat/Lng to address
# # geo_rev_data<-data %>%
# #   tidygeocoder::reverse_geocode(
# #     lat=lat,
# #     long=lng,
# #     method="osm")


# ## Calculate distance between each parking spot and each business

# # Create an empty list to store the results
# result_list <- list()

# for (i in 1:nrow(parking_data_with_nearest_business)){
  
#   # Create a temporary data frame to store the results for this parking spot
#   temp_df <- data.frame(
#     address = character(),
#     lat = double(),
#     lng = double(),
#     Convert = integer(),
#     distance = double(),
#     #capacity = integer(),
#     traffic_volume = double(),
#     #rate_half_hr= double(),
#     n_business = double(),
#     n_customers =double(),
#     time_spent = double(),
#     category = character(),
#     Operating.name = character(),
#     interaction_term = double()
# )
  
#   for( j in 1:nrow(business)){
#     lon1 <- parking_data_with_nearest_business[i,"lng"]
#     lat1 <- parking_data_with_nearest_business[i,"lat"]
#     lon2 <- business[j, "long"]
#     lat2 <- business[j, "lat"]

#     # # Calculate distance
#     distance <- haversine_distance(lon1,lat1, lon2, lat2)
    
#     # Store the results in the temporary data frame
#     temp_df[nrow(temp_df) + 1, ] <- list(
#       address = parking_data_with_nearest_business[i,"address"],
#       lat = parking_data_with_nearest_business[i,"lat"],
#       lng = parking_data_with_nearest_business[i,"lng"],
#       Convert = parking_data_with_nearest_business[i,"Convert"],
#       distance = distance,
#       #capacity = parking_data_with_nearest_business[i,"capacity"],
#       traffic_volume = parking_data_with_nearest_business[i,"traffic_volume"],
#       n_business = parking_data_with_nearest_business[i,"nearest_business"],
#       #rate_half_hr = parking_data_with_nearest_business[i,"rate_half_hr"],
#       n_customers = business[j,"qCustomer"],
#       time_spent = business[j,"tCustomer"],
#       category = business[j,"Category"],
#       Operating.name = business[j,"Operating.Name"],
#       interaction_term = business[j,"qCustomer"] * business[j,"tCustomer"]
#     )
#   }
  
#   # Append the temporary data frame to the result list
#   result_list[[i]] <- temp_df
# }

# # Combine all the results into a single data frame
# result_df <- do.call(rbind, result_list)

# library(dplyr)
# filtered_result_df2 <- result_df %>%
#   group_by(Operating.name) %>% 
#   slice(which.min(distance))

# summarized_result_df <- filtered_result_df2 %>%
#   group_by(address) %>%
#   summarize(
#     mean_lat = mean(lat),
#     mean_lng = mean(lng),
#     mean_distance = mean(distance),
#     mean_traffic_volume = mean(traffic_volume),
#     mean_n_business = mean(n_business),
#     mean_n_customers = mean(n_customers),
#     mean_time_spent = mean(time_spent),
#     mean_interaction_term = mean(interaction_term),
#     convert=mean(Convert)
#   )
# #save summarized_result_df as a csv for future ease of use
# write.csv(summarized_result_df,"../Data/DataProcessing/summarized_result_df.csv")

# # train_indices <- sample(1:nrow(parking_data]), 0.7 * nrow(parking_data))  # 70% for training
# # train_data <- parking_data[train_indices, ]
# # test_data <- parking_data[-train_indices, ]

# # Create a formula including predictor columns
# formula <- as.formula("convert ~ mean_distance + mean_traffic_volume + mean_n_business")

# # Create the SVM model with the formula
# svm_model <- svm(formula, data = summarized_result_df, kernel = "linear")

# # Coefficients from the SVM model
# coefficients <- coef(svm_model)[-1]  # Exclude the intercept

# # Select the columns for which to calculate the score
# cols <- c("mean_distance", "mean_n_business", "mean_traffic_volume")

# # Calculate the score for each row
# summarized_result_df$score <- rowSums(summarized_result_df[cols] * coefficients)
# # Sort the rows in descending order according to the score
# summarized_result_df <- summarized_result_df %>%
#   arrange(desc(score))

# # View the sorted result (Top 12 Parking Locations)
# head(summarized_result_df,9)


In [170]:

## Pre-Processing 4 : 
# Use KNN to find nearest distance, average number of customers and average time spent at business location.
# Add EV traffic volume data for year 2022 and 2024. 
# Assume business are static 
# output data will be used for training of the classification model




if(RUN_DATA_PROCESSING) {
    print("RUN_DATA_PROCESSING is TRUE")



    # Read data
    parking_data <- data
    business_data <- biz_geo_loc1

    K <- 20  # Number of nearest businesses to consider

    for (i in 1:nrow(parking_data)) {
        parking_spot <- parking_data[i, c("lat", "lng")]  # Coordinates of the parking spot
        nearest_biz_indices <- get.knnx(business_data[, c("lat", "long")], parking_spot, k = K, algorithm = "kd_tree")$nn.index
        nearest_biz_distances <- get.knnx(business_data[, c("lat", "long")], parking_spot, k = K, algorithm = "kd_tree")$nn.dist

        # Calculate the number of nearest businesses and their distances
        num_nearest_biz <- length(nearest_biz_indices)
        avg_distance <- mean(nearest_biz_distances)

        nearest_biz_customers <- sapply(nearest_biz_indices, function(idx) {
      mean(business_data[idx, "qCustomer"])
    })
        nearest_biz_time_spent <- sapply(nearest_biz_indices, function(idx) {
      mean(business_data[idx, "tCustomer"])
    })

        overall_mean_customers <- mean(nearest_biz_customers)
        overall_mean_time_spent <- mean(nearest_biz_time_spent)


        # Add the results to the parking_data dataframe
    #     parking_data[i, paste0("nearest_biz_count_", K)] <- num_nearest_biz
    #     parking_data[i, paste0("avg_distance_to_biz_", K)] <- avg_distance
    #     parking_data[i, paste0("avg_customers_nearby_", K)] <- mean(overall_mean_customers)
    #     parking_data[i, paste0("avg_time_spent_nearby_", K)] <- mean(overall_mean_time_spent)
        parking_data[i, "n_business"] <- num_nearest_biz
        parking_data[i, "distance"] <- avg_distance
        parking_data[i, "n_customers"] <- mean(overall_mean_customers)
        parking_data[i, "time_spent"] <- mean(overall_mean_time_spent)
    }


    # Add EV Traffic data from 2022 for training. 
    parking_data$traffic_volume=TS_Output_df$EV_nearbyTraffic_2022	

    # #save parking_data as a csv for future ease of use
    # write.csv(parking_data,"../Data/DataProcessing/parking_data_pre_grouping.csv")


    # # Group by address
    # parking_data <- parking_data %>%
    #   group_by(address) %>%
    #   summarise(
    #     distance = mean(avg_distance_to_biz_8),
    #     n_business = mean(nearest_biz_count_8),
    #     time_spent =mean(avg_time_spent_nearby_8),
    #     n_customers=mean(avg_customers_nearby_8),
    #     traffic_volume = mean(traffic_volume),
    #     convert = mean(Convert)
    #   )


    # Group by ID
    # parking_data <- parking_data %>%
    #   group_by(id) %>%
    #   summarise(
    # #     address = 
    #     distance = mean(avg_distance_to_biz_8),
    #     n_business = mean(nearest_biz_count_8),
    #     time_spent =mean(avg_time_spent_nearby_8),
    #     n_customers=mean(avg_customers_nearby_8),
    #     traffic_volume = mean(traffic_volume),
    #     convert = mean(Convert),
    #     capacity = mean(capacity)
    #   )


    # Normalize predictors
    # parking_data$distance <- scale(parking_data$distance)
    # parking_data$traffic_volume <- scale(parking_data$traffic_volume)
    # parking_data$time_spent<- scale(parking_data$time_spent)
    # parking_data$n_customers<- scale(parking_data$n_customers)

    # no scale
    # parking_data <- parking_data %>%
    #     mutate(distance = distance/max(distance)) %>%
    #     mutate(traffic_volume = traffic_volume/max(traffic_volume)) %>%
    #     mutate(time_spent = time_spent/max(time_spent)) %>%
    #     mutate(n_customers = n_customers/max(n_customers)) %>%
    #     mutate(capacity = capacity/max(capacity))

    #save summarized_result_df as a csv for future ease of use
    write.csv(parking_data,"../Data/DataProcessing/parking_data.csv")

} else {
    print("RUN_DATA_PROCESSING is FALSE")
    parking_data <- read.csv("../Data/DataProcessing/parking_data.csv") # read from pre-processed file
}
head(parking_data) # preview data

[1] "RUN_DATA_PROCESSING is FALSE"


,X,id,address,lat,lng,capacity,Convert,n_business,distance,n_customers,time_spent,traffic_volume
,<int>,<int>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
1,1,3,13 Isabella Street,43.66758,-79.38471,33,0,20,0.001647831,28.80,1.2391036,145
2,2,5,15 Wellesley Street East,43.66484,-79.38359,135,0,20,0.002253017,28.70,1.0503689,158
3,3,12,30 Alvin Avenue,43.68919,-79.39270,188,0,20,0.004607941,27.20,0.9009667,136
4,4,17,716 Pape Avenue,43.67971,-79.34538,70,0,20,0.002608467,25.90,1.0526270,147
5,5,18,351 Keele Street,43.66458,-79.46399,77,0,20,0.006052567,30.75,1.0121779,182
6,6,19,385 Pacific Avenue,43.66474,-79.46812,71,0,20,0.004097295,30.05,0.8493223,173


In [183]:
############################################## METHOD 2 - SVM (With KNN) ##################################################

## Train Classification and make prediction: 
# Split input data into subsets for training and prediction
# Make SVM model with 'convert' as dependent variable, and select others as independent variable
# Use decision value to rank the prediction outcome and correlate to the EV conversion priority. 
# OUTPUT: parking locations to be converted to EV charging stations in a ranked order.


set.seed(123)  # for reproducibility
# 10% for training of non-EV and 9
EV_parking <- filter(parking_data,Convert ==1)
# head(EV_parking)
Non_EV_Parking <- filter(parking_data,Convert ==0)
# head(Non_EV_Parking)

# write.csv(Non_EV_Parking,"../Data/DataProcessing/Non_EV_Parking.csv")

train_indices <- sample(1:nrow(Non_EV_Parking), 0.05 * nrow(Non_EV_Parking))  # 10% of data
Non_EV_train_data <- Non_EV_Parking[train_indices, ]
# test_data <- parking_data[-train_indices, ]

train_data <- bind_rows(EV_parking,Non_EV_train_data)
# write.csv(train_data,"../Data/DataProcessing/train_data.csv")

predict_data <- parking_data %>%
mutate(traffic_volume = TS_Output_df$EV_nearbyTraffic_2024) %>%
filter(Convert == 0)

# write.csv(predict_data,"../Data/DataProcessing/predict_data.csv")
    

# # Create a formula including predictor columns ( no simulated data)
formula <- as.formula("Convert ~ distance + traffic_volume + capacity + n_customers + time_spent")

# with simulated data
# formula <- as.formula("Convert ~ distance + traffic_volume + ")

# Create the SVM model with the formula
# svm_model <- svm(formula, data = parking_data, kernel = "linear")

svm_model <- svm(formula, data = train_data, kernel = "linear", type = "C-classification")

summary(svm_model)

pred = predict(svm_model, newdata = train_data)

pred_table = table( pred, train_data$Convert)
pred_table
sprintf("SVM Model prediction accuracy is %.2f %%",(pred_table[1,1]+pred_table[2,2])/sum(pred_table) *100)
# x = select(train_data,one_of(c("distance","traffic_volume","capacity")))
# y = select(train_data,"Convert")



# pred <- predict(svm_model, x, decision.values = TRUE)
# d_values <- attr(pred, "decision.values")[,]


# write.csv(d_values,"../Data/DataProcessing/d_values.csv")



# # Get the coefficients (weights) of the model
# weights <- coef(svm_model)
# weights
# # Coefficients from the SVM model
# coefficients <- coef(svm_model)[-1]  # Exclude the intercept

# # Select the columns for which to calculate the score
# cols <- c("distance",  "traffic_volume","capacity")

# # Calculate the score for each row
# parking_data$score <- rowSums(parking_data[cols] * coefficients)

# # Sort the rows in descending order according to the score

pred_outcome_svm = predict(svm_model, newdata = predict_data, decision.values = TRUE )
sorted_result <- predict_data
sorted_result$pred_outcome_svm = pred_outcome_svm
sorted_result$d_value = attributes(pred_outcome_svm)$decision.values
# sorted_result <- predict_data %>%
#   mutate(pred_outcome_svm = predict(svm_model, newdata = ., decision.values = TRUE )) #%>% 
#   mutate(Pred_D_value = attr(predict(svm_model, newdata = ., decision.values = TRUE ),decision.values))
sorted_result<- sorted_result %>%
    arrange(desc(d_value)) %>%
    select(id,address, d_value)

# View the sorted result (Top  parking locations)
head(sorted_result,5)

#save sorted_result as a csv for future ease of use
write.csv(sorted_result,"../Data/DataProcessing/sorted_result.csv")


Call:
svm(formula = formula, data = train_data, kernel = "linear", type = "C-classification")


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  1 

Number of Support Vectors:  15

 ( 7 8 )


Number of Classes:  2 

Levels: 
 0 1




    
pred 0 1
   0 7 1
   1 3 8

[1] "SVM Model prediction accuracy is 78.95 %"

,id,address,d_value
,<int>,<chr>,"<dbl[,1]>"
1,44,14 Fuller Avenue,1.535779
2,256,1624 Queen Street West,1.383867
3,279,287 Rushton Road,1.293679
4,220,789 St. Clair Avenue West,1.214196
5,71,35 Bellevue Avenue,1.049167


In [ ]:
# Result and Discussion

# The SVM model was constructed with data that had a lot of underlying assumptions, including some artificial
# simulated data. The intention is to demonstrate a approach translate multiple sources of data into actionable
# business decisions using  machine learning modelling. 
# The specific machine learning techniques are:
#  K Nearest Neighbor, Time Series Forecasting and SVM Classification
# The Final Model and prediction have several drawbacks. 
# 1) the training data is limited as there are only 9 parking lots with EV chargers out of 220 surface lots. 
# 2) the decision values are close to zero, which represents low confidence guesses. This reflects a rather poor model. 
# More data input such as near by population and acutal car park usage rate and real business customer data could've
# produced better output. A larger training set could also improve the performance of the model with more confident
# predictions. Overall, the approach is shown to be feasible and could yield potential benefits in complimenting the 
# decision making process. 

# see more details in the report. 
